Inspired by https://www.kaggle.com/siavrez/kerasembeddings

In [1]:
from tensorflow.keras import layers, optimizers, callbacks, utils, losses, metrics, backend as K
from sklearn import metrics as skmetrics, preprocessing
from tensorflow.keras.models import Model, load_model
from sklearn.model_selection import StratifiedKFold
from scipy.stats import rankdata
import os, gc, joblib, warnings
import tensorflow_addons as tfa
import tensorflow as tf
import pandas as pd
import numpy as np

import datetime as dt
from typing import Tuple, List, Dict

C:\ProgramData\Anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\ProgramData\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.IPBC74C7KURV7CB2PKT5Z5FNR3SIBV4J.gfortran-win_amd64.dll
C:\ProgramData\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
warnings.filterwarnings('ignore')

In [3]:
def create_model(data, catcols):    
    inputs = []
    outputs = []
    for c in catcols:
        num_unique_values = int(data[c].nunique())
        embed_dim = int(min(np.ceil((num_unique_values)/2), 20))
        inp = layers.Input(shape=(1,))
        out = layers.Embedding(num_unique_values + 1, embed_dim, name=c)(inp)
        out = layers.SpatialDropout1D(0.25)(out)
        out = layers.Reshape(target_shape=(embed_dim, ))(out)
        inputs.append(inp)
        outputs.append(out)
    
    x = layers.Concatenate()(outputs)
    x = layers.BatchNormalization()(x)
    
    x = layers.Dense(300, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    x = layers.BatchNormalization()(x)
    
    x = layers.Dense(300, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    x = layers.BatchNormalization()(x)
    
    y = layers.Dense(1, activation='sigmoid')(x)

    model = Model(inputs=inputs, outputs=y)
    return model

In [4]:
# read data
in_kaggle = False


def get_data_file_path(is_in_kaggle: bool) -> Tuple[str, str, str]:
    train_path = ''
    test_path = ''
    sample_submission_path = ''

    if is_in_kaggle:
        # running in Kaggle, inside the competition
        train_path = '../input/tabular-playground-series-mar-2021/train.csv'
        test_path = '../input/tabular-playground-series-mar-2021/test.csv'
        sample_submission_path = '../input/tabular-playground-series-mar-2021/sample_submission.csv'
    else:
        # running locally
        train_path = 'data/train.csv'
        test_path = 'data/test.csv'
        sample_submission_path = 'data/sample_submission.csv'

    return train_path, test_path, sample_submission_path

In [5]:
# main flow
start_time = dt.datetime.now()
print("Started at ", start_time)

Started at  2021-03-30 23:35:22.436382


In [6]:
%%time
# get the training set and labels
train_set_path, test_set_path, sample_subm_path = get_data_file_path(in_kaggle)

train = pd.read_csv(train_set_path)
test = pd.read_csv(test_set_path)
target = train.target

test_id = test.id

subm = pd.read_csv(sample_subm_path)

Wall time: 2.66 s


In [7]:
test_id = test.id.values
train = train.drop(['id'], axis=1)
test = test.drop(['id'], axis=1)

In [8]:
sparse_features = [f for f in train.columns if 'cat' in f]
dense_features = [feat for feat in train.columns if  feat not in sparse_features+['target']]

for col in sparse_features:
    train_only = list(set(train[col].unique()) - set(test[col].unique()))
    test_only = list(set(test[col].unique()) - set(train[col].unique()))
    both = list(set(test[col].unique()).union(set(train[col].unique())))
    train.loc[train[col].isin(train_only), col] = np.nan
    test.loc[test[col].isin(test_only), col] = np.nan
    mode = train[col].mode().values[0]
    train[col] = train[col].fillna(mode)
    test[col] = test[col].fillna(mode)
    

In [9]:
for feat in dense_features:
    test[feat] = np.clip(test[feat], train[feat].min(), train[feat].max())

In [10]:
test["target"] = -1
data = pd.concat([train, test]).reset_index(drop=True)

for c in dense_features:
    data[f'q_{c}'], bins_ = pd.qcut(data[c], 25, retbins=True, labels=[i for i in range(25)])
    data[f'q_{c}'] = data[f'q_{c}'].astype('str')
    sparse_features.append(f'q_{c}')

In [11]:
features = sparse_features
for feat in features:
    lbl_enc = preprocessing.OrdinalEncoder()
    data[feat] = lbl_enc.fit_transform(data[feat].fillna('-1').values.reshape(-1,1).astype(str))
    
train = data[data.target != -1].reset_index(drop=True)
test = data[data.target == -1].reset_index(drop=True)
test_data = [test.loc[:, features].values[:, k] for k in range(test.loc[:, features].values.shape[1])]

In [12]:
oof_preds = np.zeros((len(train)))
bagged_oof_preds = np.zeros((len(train)))
test_preds = np.zeros((len(test)))
bagged_test_preds = np.zeros((len(test)))
learning_rate = 0.00095
label_smoothing = 0.0
Verbose = 0
n_splits = [10, 15]
n_bags = 2
seeds = [2021, 2021]
n_epochs=106

In [13]:
for bag in range(n_bags):
    print(f'Iteration {bag+1} splits {n_splits[bag]} seed {seeds[bag]}')
    for fold, (train_index, test_index) in enumerate(StratifiedKFold(n_splits=n_splits[bag], shuffle=True, random_state=seeds[bag]).split(train, train.target.values)):
        X_train, X_test = train.iloc[train_index, :], train.iloc[test_index, :]
        X_train = X_train.reset_index(drop=True)
        X_test = X_test.reset_index(drop=True)
        y_train, y_test = X_train.target.values, X_test.target.values
        model = create_model(data, features)
        model.compile(
            optimizer=tfa.optimizers.SWA(tf.keras.optimizers.Adam(learning_rate=learning_rate)),
            loss=losses.BinaryCrossentropy(label_smoothing=label_smoothing),
            metrics=metrics.AUC(name="AUC"),
        )

        X_train = [X_train.loc[:, features].values[:, k] for k in range(X_train.loc[:, features].values.shape[1])]
        X_test = [X_test.loc[:, features].values[:, k] for k in range(X_test.loc[:, features].values.shape[1])]
    
        es = callbacks.EarlyStopping(monitor='val_AUC', min_delta=0.000001, patience=10, verbose=Verbose, mode='max', baseline=None, restore_best_weights=True)
        sb = callbacks.ModelCheckpoint('./nn_model.w8', save_weights_only=True, save_best_only=True, verbose=Verbose, monitor='val_AUC',mode='max')
        plateau  = callbacks.ReduceLROnPlateau(monitor='val_AUC', factor=0.5, patience=2, verbose=Verbose,
                                        mode='max', min_delta=0.0001, cooldown=0, min_lr=1e-7)
        model.fit(X_train,
                  y_train,
                  validation_data=(X_test, y_test),
                  verbose=Verbose,
                  batch_size=1024,
                  callbacks=[es, sb, plateau],
                  epochs=n_epochs
             )
        valid_fold_preds = model.predict(X_test)
        test_fold_preds = model.predict(test_data)
        oof_preds[test_index] = rankdata(valid_fold_preds.ravel())/len(X_test)
        test_preds += rankdata(test_fold_preds.ravel() / n_splits[bag])/len(test)
        print(f'fold {fold+1} AUC : {skmetrics.roc_auc_score(y_test, valid_fold_preds)}')
        K.clear_session()
    print(f'Overall AUC of Iteration {bag+1} = {skmetrics.roc_auc_score(train.target.values, oof_preds)}')
    np.save(f'oof_preds_{bag}',oof_preds)
    np.save(f'test_preds_{bag}',test_preds)
    bagged_test_preds += test_preds / n_bags
    bagged_oof_preds += oof_preds / n_bags

Iteration 1 splits 10 seed 2021
fold 1 AUC : 0.8967671473659682
fold 2 AUC : 0.8943639238159471
fold 3 AUC : 0.892243698971869
fold 4 AUC : 0.891725929405915
fold 5 AUC : 0.8940745824192687
fold 6 AUC : 0.8947464482700873
fold 7 AUC : 0.8962599489512402
fold 8 AUC : 0.8949227453991326
fold 9 AUC : 0.8909032313449967
fold 10 AUC : 0.8930278304852037
Overall AUC of Iteration 1 = 0.8939035419903132
Iteration 2 splits 15 seed 2021
fold 1 AUC : 0.895729882483057
fold 2 AUC : 0.8967605405992782
fold 3 AUC : 0.8954609877936681
fold 4 AUC : 0.8909061233581543
fold 5 AUC : 0.8950622553094599
fold 6 AUC : 0.8897915881261642
fold 7 AUC : 0.8925182010773224
fold 8 AUC : 0.8952172658689886
fold 9 AUC : 0.8966495454630724
fold 10 AUC : 0.895821306942058
fold 11 AUC : 0.8985806816986095
fold 12 AUC : 0.8928987365914883
fold 13 AUC : 0.8935752764664187
fold 14 AUC : 0.8910229469322822
fold 15 AUC : 0.8933549049625584
Overall AUC of Iteration 2 = 0.8942233475264485


In [14]:
print("Overall AUC={}".format(skmetrics.roc_auc_score(train.target.values, bagged_oof_preds)))

Overall AUC=0.8950077419461052


In [18]:
print('Saving submission file')
submission = pd.DataFrame.from_dict({
    'id': test_id,
    'target': bagged_test_preds,
})
submission.to_csv('keras_embedding_lr00095_subm.csv', index=False)
submission.head(3)

Saving submission file


,id,target
0,5,8.524240
1,6,13.794042
2,8,1.150105


In [16]:
#top_public = pd.read_csv('/kaggle/input/tps-mar-2021-stacked-starter/submission.csv')
#submission['target'] = (rankdata(submission.target) * 0.275 + rankdata(top_public.target) * 0.725)/len(submission)
#submission.to_csv('blend.csv', index=False)
#submission.head(3)

In [17]:
print('We are done. That is all, folks!')
finish_time = dt.datetime.now()
print("Finished at ", finish_time)
elapsed = finish_time - start_time
print("Elapsed time: ", elapsed)

We are done. That is all, folks!
Finished at  2021-03-31 02:30:36.028785
Elapsed time:  2:55:13.592403


Public LB Score: 0.89111